In [ ]:
import pandas as pd
data=pd.read_csv("diabetes.csv")
data.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [ ]:
data.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [ ]:
model = data.iloc[:768]

In [ ]:
len(data)

768

In [ ]:
len(model)

768

In [ ]:
model.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Split the data into features and target
X = model.drop(columns=["Outcome"])
y = model["Outcome"]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create a Gaussian Naive Bayes classifier
gnb = GaussianNB()

# Fit the model to the training data
gnb.fit(X_train, y_train)

# Make predictions on the test data
y_pred = gnb.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)


Accuracy: 0.7597402597402597


Traditional Clinical Trial

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create a Gaussian Naive Bayes classifier
gnb = GaussianNB()

# Fit the model to the training data
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
y_prob = gnb.predict_proba(X_test)

df = pd.DataFrame(y_prob, columns=["Class 0 Probability", "Class 1 Probability"])
df["Obs"] = range(1, len(df) + 1)

for i in range(len(df)):
    if df.loc[i, "Class 0 Probability"] < df.loc[i, "Class 1 Probability"]:
        df.loc[i, "Actual Class"] = 1 
    else:
        df.loc[i, "Actual Class"] = 0

df.set_index("Obs", inplace=True)
print(df)
class_counts = (df["Actual Class"].value_counts(normalize=True) * 10).round(decimals=1)
print(class_counts)


Accuracy: 0.7597402597402597
     Class 0 Probability  Class 1 Probability  Actual Class
Obs                                                        
1               0.718211             0.281789           0.0
2               0.939441             0.060559           0.0
3               0.970721             0.029279           0.0
4               0.808641             0.191359           0.0
5               0.473029             0.526971           1.0
..                   ...                  ...           ...
304             0.992963             0.007037           0.0
305             0.975498             0.024502           0.0
306             0.995971             0.004029           0.0
307             0.853746             0.146254           0.0
308             0.879324             0.120676           0.0

[308 rows x 3 columns]
0.0    6.5
1.0    3.5
Name: Actual Class, dtype: float64


Sequential Clinical Trial

In [ ]:
import numpy as np

# Set the number of stages to run
num_stages = 10

# Initialize a list to store the actual class value counts for each stage
actual_class_counts = []

for stage in range(num_stages):
    # Randomly select 10 observations from the test set
    random_indices = np.random.choice(X_test.index, size=10, replace=False)
    X_stage = X_test.loc[random_indices]
    y_stage = y_test.loc[random_indices]
    
    # Initialize counters for class 0 and class 1
    count_0 = 0
    count_1 = 0
    
    # Initialize a list to store the predicted class for each observation
    y_pred_list = []
    
    # Create a Gaussian Naive Bayes classifier
    gnb = GaussianNB()
    
    # Fit the model to the training data
    gnb.fit(X_train, y_train)
    
    # Loop through each observation in the stage
    for i in range(len(X_stage)):
        # Split the stage into a single observation
        X_single = X_stage.iloc[[i]]
        y_single = y_stage.iloc[[i]]
        
        # Make a prediction for the single observation
        y_single_pred = gnb.predict(X_single)[0]
        
        # Update the counters based on the actual class of the single observation
        if y_single.iloc[0] == 0:
            count_0 += 1
        else:
            count_1 += 1
        
        # Add the predicted class to the list
        y_pred_list.append(y_single_pred)
    
    # Calculate the percentage of each class based on the predicted values
    count_0_pred = y_pred_list.count(0)
    count_1_pred = y_pred_list.count(1)
    percent_0_pred = count_0_pred / len(y_pred_list) * 100
    percent_1_pred = count_1_pred / len(y_pred_list) * 100
    
    # Print the percentage of each class based on the predicted values
    print(f"Stage {stage+1}: Class 0 = {percent_0_pred:.2f}%, Class 1 = {percent_1_pred:.2f}%")
    
    # Add the actual class value counts to the list for this stage
    actual_class_counts.append(y_stage.value_counts())
    
# Calculate the average actual class value counts across all stages
avg_class_counts = sum(actual_class_counts) / num_stages

# Print the average actual class value counts
print(f"\nAverage Class Value Counts Across All Stages:\n{avg_class_counts}")


Stage 1: Class 0 = 70.00%, Class 1 = 30.00%
Stage 2: Class 0 = 70.00%, Class 1 = 30.00%
Stage 3: Class 0 = 70.00%, Class 1 = 30.00%
Stage 4: Class 0 = 90.00%, Class 1 = 10.00%
Stage 5: Class 0 = 70.00%, Class 1 = 30.00%
Stage 6: Class 0 = 40.00%, Class 1 = 60.00%
Stage 7: Class 0 = 80.00%, Class 1 = 20.00%
Stage 8: Class 0 = 50.00%, Class 1 = 50.00%
Stage 9: Class 0 = 50.00%, Class 1 = 50.00%
Stage 10: Class 0 = 70.00%, Class 1 = 30.00%

Average Class Value Counts Across All Stages:
0    6.8
1    3.2
Name: Outcome, dtype: float64


As we can observe, the ratio for both sequential and traditional trials are almost the same even after Traditional trial used 308 patients whereas sequential trial used only 100 patients

Comparing its effect to placebo:

In [ ]:
import numpy as np
import random
from scipy.stats import norm 

# Define simulation parameters
num_patients = 1000
num_interim_analyses = 3
p_value_threshold = 0.05

# Generate simulated data
vaccine_effect = 0.5
placebo_effect = 0.2
vaccine_probability = 0.5

# Initialize variables
vaccine_group = []
placebo_group = []
vaccine_effect_estimates = []
placebo_effect_estimates = []

# Simulate trial using sequential design
for i in range(num_patients):
    # Randomly assign patient to treatment group
    if random.random() < vaccine_probability:
        # Assign patient to vaccine group
        outcome = random.random() < (0.5 + vaccine_effect)
        vaccine_group.append(outcome)
    else:
        # Assign patient to placebo group
        outcome = random.random() < (0.5 + placebo_effect)
        placebo_group.append(outcome)
    
    # Conduct interim analysis
    if (i+1) % (num_patients // (num_interim_analyses+1)) == 0:
        # Estimate treatment effects
        vaccine_effect_estimate = np.mean(vaccine_group)
        placebo_effect_estimate = np.mean(placebo_group)
        vaccine_effect_estimates.append(vaccine_effect_estimate)
        placebo_effect_estimates.append(placebo_effect_estimate)
        
        # Calculate p-value for treatment comparison
        z_statistic = (vaccine_effect_estimate - placebo_effect_estimate) / np.sqrt((vaccine_effect_estimate * (1 - vaccine_effect_estimate) / len(vaccine_group)) + (placebo_effect_estimate * (1 - placebo_effect_estimate) / len(placebo_group)))
        p_value = 2 * (1 - norm.cdf(abs(z_statistic)))
        
        # Check if trial should stop
        if p_value < p_value_threshold:
            if vaccine_effect_estimate > placebo_effect_estimate:
                print("Vaccine is better than placebo!")
            else:
                print("Placebo is better than vaccine!")
            print("Sequential design terminated after {} patients.".format(len(vaccine_group) + len(placebo_group)))
            break

# Simulate trial using traditional design
vaccine_group = []
placebo_group = []
for i in range(num_patients):
    # Randomly assign patient to treatment group
    if random.random() < vaccine_probability:
        # Assign patient to vaccine group
        outcome = random.random() < (0.5 + vaccine_effect)
        vaccine_group.append(outcome)
    else:
        # Assign patient to placebo group
        outcome = random.random() < (0.5 + placebo_effect)
        placebo_group.append(outcome)

# Estimate treatment effects
vaccine_effect_estimate = np.mean(vaccine_group)
placebo_effect_estimate = np.mean(placebo_group)

# Calculate p-value for treatment comparison
standard_error = np.sqrt((vaccine_effect_estimate * (1 - vaccine_effect_estimate) / len(vaccine_group)) + (placebo_effect_estimate * (1 - placebo_effect_estimate) / len(placebo_group)))
z_statistic = (vaccine_effect_estimate - placebo_effect_estimate) / standard_error
p_value = 2 * (1 - norm.cdf(abs(z_statistic)))

#Check if trial should stop
if p_value < p_value_threshold:
    if vaccine_effect_estimate > placebo_effect_estimate:
        print("Vaccine is better than placebo!")
    else:
        print("Placebo is better than vaccine!")
    
else:
    print("Trial did not reach significance level.")

#Print results
print("Sequential design:")
print("Vaccine effect estimate: {:.2f}".format(vaccine_effect_estimate))
print("Placebo effect estimate: {:.2f}".format(placebo_effect_estimate))
print("Traditional design:")
print("Vaccine effect estimate: {:.2f}".format(vaccine_effect_estimate))
print("Placebo effect estimate: {:.2f}".format(placebo_effect_estimate))

Vaccine is better than placebo!
Sequential design terminated after 250 patients.
Vaccine is better than placebo!
Sequential design:
Vaccine effect estimate: 1.00
Placebo effect estimate: 0.74
Traditional design:
Vaccine effect estimate: 1.00
Placebo effect estimate: 0.74


In [ ]:
from scipy.stats import ttest_ind_from_stats
from statsmodels.stats.power import tt_ind_solve_power

#Set up parameters for the trial
n_total = 2000 # Total number of patients in the trial
n_initial = 500 # Number of patients in the initial stage
n_interim = 750 # Number of patients in each interim analysis
n_final = 1000 # Number of patients in the final stage
p_control = 0.5 # Probability of success in the control group
p_treatment = 0.55 # Probability of success in the treatment group

#Set up stopping boundaries and futility boundaries
z_alpha = norm.ppf(0.025)
z_beta = norm.ppf(0.1)
delta = p_treatment - p_control
futility_lower = delta / 2
futility_upper = -futility_lower
efficacy_lower = delta / 2 + z_alpha * np.sqrt(p_control * (1 - p_control) / n_initial)
efficacy_upper = delta / 2 + z_beta * np.sqrt(p_treatment * (1 - p_treatment) / n_total)

#Set up lists to store the results
n_list = [n_initial] # List of number of patients in each stage
result_list = [] # List of results of each stage (0 for control, 1 for treatment)

#Calculate the power for the specified parameters
#Calculate the power for the specified parameters
power = tt_ind_solve_power(effect_size=delta/2, alpha=0.025, nobs1=n_initial)
#Conduct the initial stage
results = [random.random() < p_treatment for _ in range(n_initial)]
result_list += results
n_list.append(n_interim)

#Conduct the interim stages until futility or efficacy is reached
while n_list[-1] < n_total:
    interim_results = [random.random() < p_treatment for _ in range(n_interim)]
    result_list += interim_results
    interim_success_rate = np.mean(interim_results)
    if interim_success_rate <= futility_lower or interim_success_rate >= futility_upper:
        print(f"Interim analysis: futility boundary crossed, trial stopped with {len(result_list)} patients")
        break
    elif interim_success_rate >= efficacy_upper:
        print(f"Interim analysis: efficacy boundary crossed, trial stopped with {len(result_list)} patients")
        
final_results = [random.random() < p_treatment for _ in range(n_final)]
result_list += final_results

#Calculate the success rate and determine if the trial was successful
success_rate = np.mean(result_list)
if success_rate <= futility_lower:
    print(f"Trial failed: success rate ({success_rate:.3f}) is below the futility boundary ({futility_lower:.3f})")
elif success_rate >= efficacy_upper:
    print(f"Trial successful: success rate ({success_rate:.3f}) is above the efficacy boundary ({efficacy_upper:.3f})")
else:
    print(f"Trial inconclusive: success rate ({success_rate:.3f}) is between the futility and efficacy boundaries")

#Print the results of the trial
print(f"Number of patients: {n_total}")
print(f"Number of patients in each stage: {n_list}")
print(f"Success rate: {success_rate:.3f}")
print(f"Power: {power:.3f}")

Interim analysis: futility boundary crossed, trial stopped with 1250 patients
Trial successful: success rate (0.540) is above the efficacy boundary (0.011)
Number of patients: 2000
Number of patients in each stage: [500, 750]
Success rate: 0.540
Power: 0.037
